##Model

#Tracking with Sort

###Installation

In [ ]:
# ====================================================
# ✅ Minimal SORT + YOLOv8 setup (Colab)
# ====================================================

# 0️⃣ Upgrade pip
!pip install --upgrade pip

# 1️⃣ Install YOLOv8
!pip install ultralytics --upgrade

# 2️⃣ Install SORT dependencies
!pip install filterpy scipy opencv-python-headless numpy --quiet


###Sort Installations

In [ ]:
!pip install filterpy opencv-python numpy


In [ ]:
# 1. Clean clone
!rm -rf sort
!git clone https://github.com/abewley/sort.git
!pip install filterpy

# 2. Remove pyplot imports and TkAgg backend
!sed -i '/import matplotlib/d' sort/sort.py
!sed -i '/plt./d' sort/sort.py

# 3. Also remove 'TkAgg' backend forcing
!sed -i "/matplotlib.use('TkAgg')/d" sort/sort.py

import os
os.environ["MPLBACKEND"] = "Agg"   # safe backend

from sort.sort import Sort
import numpy as np

tracker = Sort()
dets = np.array([[100,100,200,200,0.9]])
print("Tracks:", tracker.update(dets))


Cloning into 'sort'...
remote: Enumerating objects: 208, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 208 (delta 45), reused 40 (delta 40), pack-reused 159 (from 1)
Receiving objects: 100% (208/208), 1.20 MiB | 29.34 MiB/s, done.
Resolving deltas: 100% (76/76), done.
Tracks: [[        100         100         200         200           2]]


###Imports

In [ ]:

from ultralytics import YOLO

import cv2
import numpy as np
from yolox.tracker.byte_tracker import BYTETracker
import numpy as np
np.float = float

ModuleNotFoundError: No module named 'yolox'

###import model from drive

In [ ]:


model1 = "/content/runs/detect/train2/weights/best.pt"
drive_destination = "/content/drive/MyDrive/Colab Notebooks/Models/yolov8n_final.pt"


### Tracker Setup

In [ ]:
from google.colab import files
uploaded = files.upload()

video_path = list(uploaded.keys())[0]


## Sort

In [ ]:
# ====================================================
# ✅ Improved SORT Tracking for Pipelines
# ====================================================
from collections import defaultdict, deque
import numpy as np
import cv2
from sort.sort import Sort   # make sure abewley/sort is cloned

model = YOLO("/content/best (5) (1).pt")

# --- Parameters ---
#PIPELINE_CLASS_ID = 0  # ⚠️ Adjust based on your YOLO class index
SMOOTHING_WINDOW = 5   # number of frames for moving average

# --- Tracker Setup ---
tracker = Sort(
    max_age=15,        # allow longer gaps
    min_hits=2,        # quicker initialization
    iou_threshold=0.1  # tolerant IOU for elongated pipes
)

# --- Track history for smoothing ---
track_history = defaultdict(lambda: deque(maxlen=SMOOTHING_WINDOW))

# ====================================================
# 4) Video Input/Output
# ====================================================
video_path = "/content/WhatsApp Vidéo 2025-07-18 à 16.01.52_5fb2d07c.mp4"  # 👈 change if needed
cap = cv2.VideoCapture(video_path)

w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS) or 30.0

out = cv2.VideoWriter("sort_pipeline_output.mp4", cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

# ====================================================
# 5) Tracking Loop
# ====================================================
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLO detection
    results = model(frame, imgsz=640, conf=0.3, verbose=False)[0]

    # Convert YOLO output to [x1, y1, x2, y2, score]
    dets = []
    if len(results.boxes):
        xyxy = results.boxes.xyxy.cpu().numpy()
        conf = results.boxes.conf.cpu().numpy()
        cls = results.boxes.cls.cpu().numpy().astype(int)

        for (x1, y1, x2, y2), s in zip(xyxy, conf):
          dets.append([x1, y1, x2, y2, s])  # keep all detections (since all = pipeline)

    dets = np.array(dets) if len(dets) else np.empty((0, 5))

    # Update SORT
    tracks = tracker.update(dets)

    # Draw tracks with smoothing
    for x1, y1, x2, y2, track_id in tracks:
        box = np.array([x1, y1, x2, y2])
        track_history[track_id].append(box)

        # --- Smooth bounding box ---
        smoothed_box = np.mean(track_history[track_id], axis=0).astype(int)
        x1, y1, x2, y2 = smoothed_box

        # Draw smoothed box
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f"Pipe ID {int(track_id)}", (x1, y1 - 6),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    out.write(frame)

cap.release()
out.release()
print("✅ SORT pipeline tracking finished. Output saved as sort_pipeline_output.mp4")


In [ ]:
# ====================================================
# ✅ Final SORT Tracking (Pipeline Only)
# ====================================================
from collections import defaultdict, deque
import numpy as np
import cv2
from sort.sort import Sort   # make sure abewley/sort is cloned

# --- Parameters ---
SMOOTHING_WINDOW = 5   # number of frames for moving average

# --- Tracker Setup ---
tracker = Sort(
    max_age=15,        # allow longer gaps if detection drops
    min_hits=2,        # quicker initialization
    iou_threshold=0.1  # tolerant IOU for elongated pipes
)

# --- Track history for smoothing ---
track_history = defaultdict(lambda: deque(maxlen=SMOOTHING_WINDOW))

# ====================================================
# 4) Video Input/Output
# ====================================================
video_path = "/content/WhatsApp Vidéo 2025-07-18 à 16.01.52_5fb2d07c.mp4"  # 👈 change to your input video
cap = cv2.VideoCapture(video_path)

w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS) or 30.0

out = cv2.VideoWriter("sort_pipeline_output1.mp4", cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

# ====================================================
# 5) Tracking Loop
# ====================================================
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLO detection (single class: pipeline)
    results = model(frame, imgsz=640, conf=0.3, verbose=False)[0]

    # Convert YOLO output to [x1, y1, x2, y2, score]
    dets = []
    if len(results.boxes):
        xyxy = results.boxes.xyxy.cpu().numpy()
        conf = results.boxes.conf.cpu().numpy()

        for (x1, y1, x2, y2), s in zip(xyxy, conf):
            dets.append([x1, y1, x2, y2, s])

    dets = np.array(dets) if len(dets) else np.empty((0, 5))

    # Update SORT
    tracks = tracker.update(dets)

    # Draw tracks with smoothing
    for x1, y1, x2, y2, track_id in tracks:
        box = np.array([x1, y1, x2, y2])
        track_history[track_id].append(box)

        # --- Smooth bounding box ---
        smoothed_box = np.mean(track_history[track_id], axis=0).astype(int)
        x1, y1, x2, y2 = smoothed_box

        # Draw smoothed box
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f"Pipe ID {int(track_id)}", (x1, y1 - 6),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    out.write(frame)

cap.release()
out.release()
print("✅ SORT pipeline tracking finished. Output saved as sort_pipeline_output.mp4")


In [ ]:
from google.colab import files
files.download("sort_pipeline_output.mp4")


In [ ]:
# ====================================================
# ✅ SORT Tracking with Smoothed Red Trajectory for Drone
# ====================================================
from collections import defaultdict, deque
import numpy as np
import cv2
import math
from sort.sort import Sort
from ultralytics import YOLO

# ----------------------
# Load YOLO model (pipeline only)
# ----------------------
model = YOLO("/content/best (5) (1).pt")

# ----------------------
# Parameters
# ----------------------
SMOOTHING_WINDOW = 5       # box smoothing
TRAJECTORY_WINDOW = 10     # trajectory smoothing
CONF_THRESHOLD = 0.3

# ----------------------
# Tracker Setup
# ----------------------
tracker = Sort(max_age=15, min_hits=2, iou_threshold=0.1)
track_history = defaultdict(lambda: deque(maxlen=SMOOTHING_WINDOW))
trajectory_history = deque(maxlen=TRAJECTORY_WINDOW)
angle_history = deque(maxlen=TRAJECTORY_WINDOW)

# ----------------------
# Video Paths
# ----------------------
video_path = "/content/WhatsApp Vidéo 2025-07-18 à 16.23.28_d90beaab.mp4"
output_video_path = "sort_pipeline_output_new.mp4"
output_txt_path = "pipeline_line_data.txt"

cap = cv2.VideoCapture(video_path)
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS) or 30.0

out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

# ----------------------
# Frame Data Storage
# ----------------------
frame_data = []

# ====================================================
# Tracking Loop
# ====================================================
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # --- YOLO Detection ---
    results = model(frame, imgsz=640, conf=CONF_THRESHOLD, verbose=False)[0]
    dets = []
    if len(results.boxes):
        xyxy = results.boxes.xyxy.cpu().numpy()
        confs = results.boxes.conf.cpu().numpy()
        for (x1, y1, x2, y2), s in zip(xyxy, confs):
            dets.append([x1, y1, x2, y2, s])
    dets = np.array(dets) if len(dets) else np.empty((0, 5))

    # --- Update SORT ---
    tracks = tracker.update(dets)
    frame_mid_lines = []

    for x1, y1, x2, y2, track_id in tracks:
        # --- Smooth box ---
        track_history[track_id].append(np.array([x1, y1, x2, y2]))
        smoothed_box = np.mean(track_history[track_id], axis=0).astype(int)
        x1, y1, x2, y2 = smoothed_box

        # --- Draw bounding box ---
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f"Pipe ID {int(track_id)}", (x1, y1 - 6),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

        # --- Compute mid-line ---
        if (x2 - x1) > (y2 - y1):  # horizontal-ish pipe
            mid_y = (y1 + y2) // 2
            start_point = (x1, mid_y)
            end_point = (x2, mid_y)
        else:  # vertical-ish pipe
            mid_x = (x1 + x2) // 2
            start_point = (mid_x, y1)
            end_point = (mid_x, y2)

        frame_mid_lines.append((start_point, end_point))

    # --- Compute smoothed trajectory (RED) ---
    if frame_mid_lines:
        avg_start = (
            int(sum(pt[0][0] for pt in frame_mid_lines) / len(frame_mid_lines)),
            int(sum(pt[0][1] for pt in frame_mid_lines) / len(frame_mid_lines))
        )
        avg_end = (
            int(sum(pt[1][0] for pt in frame_mid_lines) / len(frame_mid_lines)),
            int(sum(pt[1][1] for pt in frame_mid_lines) / len(frame_mid_lines))
        )

        trajectory_history.append((avg_start, avg_end))

        smoothed_start = (
            int(sum(pt[0][0] for pt in trajectory_history) / len(trajectory_history)),
            int(sum(pt[0][1] for pt in trajectory_history) / len(trajectory_history))
        )
        smoothed_end = (
            int(sum(pt[1][0] for pt in trajectory_history) / len(trajectory_history)),
            int(sum(pt[1][1] for pt in trajectory_history) / len(trajectory_history))
        )

        # --- Compute smoothed angle ---
        dx = smoothed_end[0] - smoothed_start[0]
        dy = smoothed_end[1] - smoothed_start[1]
        angle_deg = math.degrees(math.atan2(dy, dx))
        angle_history.append(angle_deg)
        smoothed_angle = sum(angle_history) / len(angle_history)

        # --- Draw smoothed trajectory (RED) ---
        cv2.line(frame, smoothed_start, smoothed_end, (0, 0, 255), 3)

        # --- Save frame data ---
        frame_data.append((smoothed_start, smoothed_end, dx, dy, smoothed_angle))

        print(f"Frame {int(cap.get(cv2.CAP_PROP_POS_FRAMES))}: "
              f"Start={smoothed_start}, End={smoothed_end}, Δx={dx}, Δy={dy}, Angle={smoothed_angle:.2f}°")

    out.write(frame)

cap.release()
out.release()

# ====================================================
# Save Line Data to TXT
# ====================================================
with open(output_txt_path, "w") as f:
    for idx, (start, end, dx, dy, angle) in enumerate(frame_data, 1):
        f.write(f"Frame {idx}: Start={start}, End={end}, Δx={dx}, Δy={dy}, Angle={angle:.2f}°\n")

print("✅ SORT tracking complete. Output saved at", output_video_path)
print(f"Full pipeline line coordinates + deviation saved to {output_txt_path}")


Smooth

In [ ]:
# ====================================================
# ✅ SORT Tracking with Drone-Friendly Smooth Trajectory
# ====================================================
from collections import defaultdict, deque
import numpy as np
import cv2
import math
from sort.sort import Sort
from ultralytics import YOLO

# ----------------------
# Load YOLO model (pipeline only)
# ----------------------
model = YOLO("/content/drive/MyDrive/Colab Notebooks/Models/yolov8n_final.pt")

# ----------------------
# Parameters
# ----------------------
SMOOTHING_WINDOW = 5      # box smoothing
TRAJECTORY_WINDOW = 10    # trajectory smoothing for drone
CONF_THRESHOLD = 0.3

# ----------------------
# Tracker Setup
# ----------------------
tracker = Sort(
    max_age=15,
    min_hits=2,
    iou_threshold=0.1
)

track_history = defaultdict(lambda: deque(maxlen=SMOOTHING_WINDOW))
trajectory_history = deque(maxlen=TRAJECTORY_WINDOW)
angle_history = deque(maxlen=TRAJECTORY_WINDOW)

# ----------------------
# Video Paths
# ----------------------
video_path = "/content/WhatsApp Vidéo 2025-07-18 à 16.01.52_5fb2d07c.mp4"
output_video_path = "sort_pipeline_drone_output_smooth.mp4"
output_txt_path = "pipeline_drone_line_data_smooth.txt"

cap = cv2.VideoCapture(video_path)
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS) or 30.0

out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

# ----------------------
# Frame Data Storage
# ----------------------
frame_data = []

# ====================================================
# Tracking Loop
# ====================================================
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # --- YOLO Detection ---
    results = model(frame, imgsz=640, conf=CONF_THRESHOLD, verbose=False)[0]
    dets = []
    if len(results.boxes):
        xyxy = results.boxes.xyxy.cpu().numpy()
        confs = results.boxes.conf.cpu().numpy()
        for (x1, y1, x2, y2), s in zip(xyxy, confs):
            dets.append([x1, y1, x2, y2, s])
    dets = np.array(dets) if len(dets) else np.empty((0, 5))

    # --- Update SORT ---
    tracks = tracker.update(dets)
    frame_mid_lines = []

    for x1, y1, x2, y2, track_id in tracks:
        # --- Smooth box ---
        track_history[track_id].append(np.array([x1, y1, x2, y2]))
        smoothed_box = np.mean(track_history[track_id], axis=0).astype(int)
        x1, y1, x2, y2 = smoothed_box

        # --- Draw bounding box ---
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f"Pipe ID {int(track_id)}", (x1, y1 - 6),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

        # --- Compute mid-line ---
        if (x2 - x1) > (y2 - y1):  # horizontal-ish pipe
            mid_y = (y1 + y2) // 2
            start_point = (x1, mid_y)
            end_point = (x2, mid_y)
        else:  # vertical-ish pipe
            mid_x = (x1 + x2) // 2
            start_point = (mid_x, y1)
            end_point = (mid_x, y2)

        cv2.line(frame, start_point, end_point, (255, 255, 255), 2)
        frame_mid_lines.append((start_point, end_point))

    # --- Compute smoothed trajectory ---
    if frame_mid_lines:
        avg_start = (
            int(sum(pt[0][0] for pt in frame_mid_lines) / len(frame_mid_lines)),
            int(sum(pt[0][1] for pt in frame_mid_lines) / len(frame_mid_lines))
        )
        avg_end = (
            int(sum(pt[1][0] for pt in frame_mid_lines) / len(frame_mid_lines)),
            int(sum(pt[1][1] for pt in frame_mid_lines) / len(frame_mid_lines))
        )

        trajectory_history.append((avg_start, avg_end))

        smoothed_start = (
            int(sum(pt[0][0] for pt in trajectory_history) / len(trajectory_history)),
            int(sum(pt[0][1] for pt in trajectory_history) / len(trajectory_history))
        )
        smoothed_end = (
            int(sum(pt[1][0] for pt in trajectory_history) / len(trajectory_history)),
            int(sum(pt[1][1] for pt in trajectory_history) / len(trajectory_history))
        )

        # --- Compute smoothed angle ---
        dx = smoothed_end[0] - smoothed_start[0]
        dy = smoothed_end[1] - smoothed_start[1]
        angle_deg = math.degrees(math.atan2(dy, dx))
        angle_history.append(angle_deg)
        smoothed_angle = sum(angle_history) / len(angle_history)

        # --- Draw smoothed trajectory ---
        cv2.line(frame, smoothed_start, smoothed_end, (0, 0, 255), 3)

        # --- Save frame data ---
        frame_data.append((smoothed_start, smoothed_end, dx, dy, smoothed_angle))

        print(f"Frame {int(cap.get(cv2.CAP_PROP_POS_FRAMES))}: "
              f"Start={smoothed_start}, End={smoothed_end}, Δx={dx}, Δy={dy}, Angle={smoothed_angle:.2f}°")

        # --- Optional: predicted next point for drone ---
        if len(trajectory_history) > 1:
            last_start, last_end = trajectory_history[-2]
            predicted_start = (smoothed_start[0] + (smoothed_start[0]-last_start[0]),
                               smoothed_start[1] + (smoothed_start[1]-last_start[1]))
            predicted_end = (smoothed_end[0] + (smoothed_end[0]-last_end[0]),
                             smoothed_end[1] + (smoothed_end[1]-last_end[1]))
            # This can be sent to drone as next target
            # send_to_drone(predicted_start, predicted_end, smoothed_angle)

    out.write(frame)

cap.release()
out.release()

# ----------------------
# Save trajectory data to TXT
# ----------------------
with open(output_txt_path, "w") as f:
    for idx, (start, end, dx, dy, angle) in enumerate(frame_data, 1):
        f.write(f"Frame {idx}: Start={start}, End={end}, Δx={dx}, Δy={dy}, Angle={angle:.2f}°\n")

print("✅ SORT tracking with drone trajectory smoothing complete.")
print("Output video:", output_video_path)
print("Trajectory data TXT:", output_txt_path)


+ prediction

In [ ]:
# ====================================================
# ✅ SORT Tracking with Drone-Friendly Smoothed Waypoints
# ====================================================
from collections import defaultdict, deque
import numpy as np
import cv2
import math
from sort.sort import Sort
from ultralytics import YOLO

# ----------------------
# Load YOLO model (pipeline only)
# ----------------------
model = YOLO("/content/best (5) (1).pt")

# ----------------------
# Parameters
# ----------------------
SMOOTHING_WINDOW = 5       # box smoothing
TRAJECTORY_WINDOW = 10     # trajectory smoothing
CONF_THRESHOLD = 0.3
PREDICTION_SCALE = 1.5     # scale how far ahead to predict next waypoint

# ----------------------
# Tracker Setup
# ----------------------
tracker = Sort(max_age=15, min_hits=2, iou_threshold=0.1)
track_history = defaultdict(lambda: deque(maxlen=SMOOTHING_WINDOW))
trajectory_history = deque(maxlen=TRAJECTORY_WINDOW)
angle_history = deque(maxlen=TRAJECTORY_WINDOW)

# ----------------------
# Video Paths
# ----------------------
video_path = "/content/WhatsApp Vidéo 2025-07-18 à 16.01.52_5fb2d07c.mp4"
output_video_path = "sort_pipeline_drone_output.mp4"
output_txt_path = "pipeline_drone_line_data.txt"

cap = cv2.VideoCapture(video_path)
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS) or 30.0

out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

# ----------------------
# Frame Data Storage
# ----------------------
frame_data = []

# ====================================================
# Tracking Loop
# ====================================================
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # --- YOLO Detection ---
    results = model(frame, imgsz=640, conf=CONF_THRESHOLD, verbose=False)[0]
    dets = []
    if len(results.boxes):
        xyxy = results.boxes.xyxy.cpu().numpy()
        confs = results.boxes.conf.cpu().numpy()
        for (x1, y1, x2, y2), s in zip(xyxy, confs):
            dets.append([x1, y1, x2, y2, s])
    dets = np.array(dets) if len(dets) else np.empty((0, 5))

    # --- Update SORT ---
    tracks = tracker.update(dets)
    frame_mid_lines = []

    for x1, y1, x2, y2, track_id in tracks:
        # --- Smooth box ---
        track_history[track_id].append(np.array([x1, y1, x2, y2]))
        smoothed_box = np.mean(track_history[track_id], axis=0).astype(int)
        x1, y1, x2, y2 = smoothed_box

        # --- Draw bounding box ---
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f"Pipe ID {int(track_id)}", (x1, y1 - 6),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

        # --- Compute mid-line ---
        if (x2 - x1) > (y2 - y1):  # horizontal-ish pipe
            mid_y = (y1 + y2) // 2
            start_point = (x1, mid_y)
            end_point = (x2, mid_y)
        else:  # vertical-ish pipe
            mid_x = (x1 + x2) // 2
            start_point = (mid_x, y1)
            end_point = (mid_x, y2)

        frame_mid_lines.append((start_point, end_point))

    # --- Compute smoothed trajectory ---
    if frame_mid_lines:
        avg_start = (
            int(sum(pt[0][0] for pt in frame_mid_lines) / len(frame_mid_lines)),
            int(sum(pt[0][1] for pt in frame_mid_lines) / len(frame_mid_lines))
        )
        avg_end = (
            int(sum(pt[1][0] for pt in frame_mid_lines) / len(frame_mid_lines)),
            int(sum(pt[1][1] for pt in frame_mid_lines) / len(frame_mid_lines))
        )

        trajectory_history.append((avg_start, avg_end))

        smoothed_start = (
            int(sum(pt[0][0] for pt in trajectory_history) / len(trajectory_history)),
            int(sum(pt[0][1] for pt in trajectory_history) / len(trajectory_history))
        )
        smoothed_end = (
            int(sum(pt[1][0] for pt in trajectory_history) / len(trajectory_history)),
            int(sum(pt[1][1] for pt in trajectory_history) / len(trajectory_history))
        )

        # --- Compute smoothed angle ---
        dx = smoothed_end[0] - smoothed_start[0]
        dy = smoothed_end[1] - smoothed_start[1]
        angle_deg = math.degrees(math.atan2(dy, dx))
        angle_history.append(angle_deg)
        smoothed_angle = sum(angle_history) / len(angle_history)

        # --- Draw smoothed trajectory (RED) ---
        cv2.line(frame, smoothed_start, smoothed_end, (0, 0, 255), 3)

        # --- Predict next waypoint for drone ---
        if len(trajectory_history) > 1:
            last_start, last_end = trajectory_history[-2]
            pred_start = (
                int(smoothed_start[0] + (smoothed_start[0] - last_start[0]) * PREDICTION_SCALE),
                int(smoothed_start[1] + (smoothed_start[1] - last_start[1]) * PREDICTION_SCALE)
            )
            pred_end = (
                int(smoothed_end[0] + (smoothed_end[0] - last_end[0]) * PREDICTION_SCALE),
                int(smoothed_end[1] + (smoothed_end[1] - last_end[1]) * PREDICTION_SCALE)
            )

            # Optional: draw predicted waypoint (cyan)
            cv2.line(frame, pred_start, pred_end, (255, 255, 0), 2)

        # --- Save frame data ---
        frame_data.append((smoothed_start, smoothed_end, dx, dy, smoothed_angle))

        print(f"Frame {int(cap.get(cv2.CAP_PROP_POS_FRAMES))}: "
              f"Start={smoothed_start}, End={smoothed_end}, Δx={dx}, Δy={dy}, Angle={smoothed_angle:.2f}°")

    out.write(frame)

cap.release()
out.release()

# ====================================================
# Save Line Data to TXT
# ====================================================
with open(output_txt_path, "w") as f:
    for idx, (start, end, dx, dy, angle) in enumerate(frame_data, 1):
        f.write(f"Frame {idx}: Start={start}, End={end}, Δx={dx}, Δy={dy}, Angle={angle:.2f}°\n")

print("✅ SORT tracking with predicted drone waypoints complete.")
print("Output video:", output_video_path)
print("Trajectory + predicted waypoints saved to:", output_txt_path)


#Bot Sort

In [ ]:
# Clone this specific BoT-SORT repo
!git clone https://github.com/30-A/botsort.git /content/botsort

# Install requirements
!pip install -r /content/botsort/requirements.txt

# Install YOLOv8
!pip install ultralytics


Cloning into '/content/botsort'...
remote: Enumerating objects: 42, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 42 (delta 16), reused 31 (delta 9), pack-reused 0 (from 0)
Receiving objects: 100% (42/42), 66.60 KiB | 16.65 MiB/s, done.
Resolving deltas: 100% (16/16), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 76.0 MB/s  0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
ERROR: Ignored the following versions that require a different python version: 1.10.0 Requires-Python >=3.8,<3.12; 1.10.0rc1 Requires-Python >=3.8,<3.12; 1.10.0rc2 Requires-Python >=3.8,<3.12; 1.10.1 Requires-Python >=3.8,<3.12; 1.21.2 Requires-Python >=3.7,<3.11; 1.21.3 Requires-Python >=3.7,<3.11; 1.21.4 Requires-Python >=3.7,<3.11; 1.21.5 Requires-Python >=3.7,<3.11; 1.21.6 Requires-Python >=3.7,<3.11; 1.6.2 Requires-Python >=3.7,<3.10; 1.6.3 Requires-Python >=3.7,<3.10; 1.7.0 Requires-Pyth

In [ ]:
import sys
sys.path.append("/content/botsort")


In [ ]:
!pip install cython_bbox lapx


  Using cached cython_bbox-0.1.5.tar.gz (4.4 kB)
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 56.2 MB/s  0:00:00
  DEPRECATION: Building 'cython_bbox' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'cython_bbox'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  Created wheel for cython_bbox: filename=cython_bbox-0.1.5-cp312-cp312-linux_x86_64.whl size=111561 sha256=a984acd74d1c1f027ae8691e0e6340e27627f9d52f9428121de78d908fc12f78
  Stored in directory: /root/.cache/pip/wheels/f1/e7/0a/7c310ac8921f2c1e61e58f80585f1ce0be9e4a36d280771857
Successfully built cython_bbox
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [cython_bbo

In [ ]:
import torch
import cv2
import numpy as np
from ultralytics import YOLO
from botsort.bot_sort import BoTSORT  # main tracker
from types import SimpleNamespace
from google.colab.patches import cv2_imshow


In [ ]:

# YOLOv8 weights
VIDEO_PATH = "/content/WhatsApp Vidéo 2025-07-18 à 16.23.28_d90beaab.mp4"
OUTPUT_PATH = "/content/tracked_Bot4.mp4"
PIPELINE_CLASS = "pipeline"

YOLO_WEIGHTS = "/content/yolov8.pt"
model = YOLO(YOLO_WEIGHTS)





In [ ]:
OUTPUT_PATH = "/content/tracked_Bot4.mp4"
PIPELINE_CLASS = "pipeline"
YOLO_WEIGHTS = "/content/best (2) (1).pt"
model = YOLO(YOLO_WEIGHTS)
# Map class
PIPE_ID =0
print(f"✅ Tracking only '{PIPELINE_CLASS}' (id={PIPE_ID})")

✅ Tracking only 'pipeline' (id=0)


In [ ]:

from collections import deque
import math

# -----------------------------
# BoT-SORT tracker setup
# -----------------------------
tracker_args = SimpleNamespace(
    track_high_thresh=0.5,
    track_low_thresh=0.1,
    new_track_thresh=0.7,
    track_buffer=30,
    match_thresh=0.8,
    use_reid=False,
    with_gmc=True,
    gmc_method="orb",
    proximity_thresh=50,
    cmc_method="orb"
)
tracker = BoTSORT(args=tracker_args, frame_rate=30)
print("✅ BoT-SORT tracker initialized")

# -----------------------------
# Video I/O
# -----------------------------
cap = cv2.VideoCapture(VIDEO_PATH)
W, H = int(cap.get(3)), int(cap.get(4))
fps = cap.get(cv2.CAP_PROP_FPS) or 30.0
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
writer = cv2.VideoWriter(OUTPUT_PATH, fourcc, fps, (W, H))

# -----------------------------
# Trajectory storage
# -----------------------------
trajectory_history = deque(maxlen=10)  # smooth last 10 frames
angle_history = deque(maxlen=10)
frame_data = []

frame_count = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame_count += 1

    # YOLOv8 detection
    results = model.predict(
        frame, imgsz=960, conf=0.25, iou=0.5,
        classes=[PIPE_ID], device=device, verbose=False
    )[0]

    # Build detections: [x1, y1, x2, y2, score]
    dets = []
    if results.boxes is not None:
        xyxy = results.boxes.xyxy.cpu().numpy()
        conf = results.boxes.conf.cpu().numpy()
        for (x1, y1, x2, y2), c in zip(xyxy, conf):
            dets.append([x1, y1, x2, y2, c])
    dets_np = np.array(dets, dtype=np.float32) if len(dets) else np.empty((0,5), dtype=np.float32)
    if dets_np.shape[0] > 0:
        class_col = np.zeros((dets_np.shape[0], 1), dtype=np.float32)  # single class = 0
        dets_np = np.hstack((dets_np, class_col))

    # Update tracker
    tracks = tracker.update(dets_np, frame)

    # --- Compute mid-lines and smoothed trajectory ---
    frame_mid_lines = []
    for t in tracks:
        x1, y1, x2, y2 = map(int, t.tlbr)
        tid = int(t.track_id)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f"pipeline#{tid}", (x1, max(0, y1-7)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        # --- Compute mid-line ---
        if (x2 - x1) > (y2 - y1):  # horizontal-ish pipe
            mid_y = (y1 + y2) // 2
            start_point = (x1, mid_y)
            end_point = (x2, mid_y)
        else:  # vertical-ish pipe
            mid_x = (x1 + x2) // 2
            start_point = (mid_x, y1)
            end_point = (mid_x, y2)

        cv2.line(frame, start_point, end_point, (255, 255, 255), 2)
        frame_mid_lines.append((start_point, end_point))

    # --- Compute smoothed trajectory ---
    if frame_mid_lines:
        avg_start = (
            int(sum(pt[0][0] for pt in frame_mid_lines) / len(frame_mid_lines)),
            int(sum(pt[0][1] for pt in frame_mid_lines) / len(frame_mid_lines))
        )
        avg_end = (
            int(sum(pt[1][0] for pt in frame_mid_lines) / len(frame_mid_lines)),
            int(sum(pt[1][1] for pt in frame_mid_lines) / len(frame_mid_lines))
        )

        trajectory_history.append((avg_start, avg_end))

        smoothed_start = (
            int(sum(pt[0][0] for pt in trajectory_history) / len(trajectory_history)),
            int(sum(pt[0][1] for pt in trajectory_history) / len(trajectory_history))
        )
        smoothed_end = (
            int(sum(pt[1][0] for pt in trajectory_history) / len(trajectory_history)),
            int(sum(pt[1][1] for pt in trajectory_history) / len(trajectory_history))
        )

        # --- Compute smoothed angle ---
        dx = smoothed_end[0] - smoothed_start[0]
        dy = smoothed_end[1] - smoothed_start[1]
        angle_deg = math.degrees(math.atan2(dy, dx))
        angle_history.append(angle_deg)
        smoothed_angle = sum(angle_history) / len(angle_history)

        # --- Draw smoothed trajectory ---
        cv2.line(frame, smoothed_start, smoothed_end, (0, 0, 255), 3)

        # --- Save frame data ---
        frame_data.append((smoothed_start, smoothed_end, dx, dy, smoothed_angle))

        print(f"Frame {frame_count}: Start={smoothed_start}, End={smoothed_end}, Δx={dx}, Δy={dy}, Angle={smoothed_angle:.2f}°")

        # --- Optional: predicted next point for drone ---
        if len(trajectory_history) > 1:
            last_start, last_end = trajectory_history[-2]
            predicted_start = (smoothed_start[0] + (smoothed_start[0]-last_start[0]),
                               smoothed_start[1] + (smoothed_start[1]-last_start[1]))
            predicted_end = (smoothed_end[0] + (smoothed_end[0]-last_end[0]),
                             smoothed_end[1] + (smoothed_end[1]-last_end[1]))
            # This can be sent to drone as next target
            # send_to_drone(predicted_start, predicted_end, smoothed_angle)

    writer.write(frame)

cap.release()
writer.release()
print(f"🎉 Done! Output saved at: {OUTPUT_PATH}")


✅ BoT-SORT tracker initialized


NameError: name 'device' is not defined

test2 faster

In [ ]:
#!/usr/bin/env python3
# ============================================
# YOLOv8 + BoT-SORT Video Processing (Save Only)
# ============================================

import cv2
import numpy as np
from ultralytics import YOLO
from botsort.bot_sort import BoTSORT
from types import SimpleNamespace

# -----------------------------
# Config
# -----------------------------
VIDEO_PATH = "/content/WhatsApp Vidéo 2025-07-18 à 16.23.28_d90beaab.mp4"        # path to input video
OUTPUT_PATH = "output.mp4"      # path to saved output
PIPE_ID = 0                     # class ID for pipeline
CONF_THRES = 0.25
IOU_THRES = 0.5

device = "cuda" if cv2.cuda.getCudaEnabledDeviceCount() > 0 else "cpu"
half = (device == "cuda")

# -----------------------------
# Load YOLOv8 model
# -----------------------------
model = YOLO("/content/best (2) (1).pt").to(device)
print(f"✅ YOLOv8 model loaded on {device}")

# -----------------------------
# Initialize BoT-SORT tracker
# -----------------------------
tracker_args = SimpleNamespace(
    track_high_thresh=0.5,
    track_low_thresh=0.1,
    new_track_thresh=0.7,
    track_buffer=30,
    match_thresh=0.8,
    proximity_thresh=50,       # required
    appearance_thresh=0.25,    # required
    cmc_method="sparseOptFlow",
    frame_rate=30,
    lambda_=0.98,
    mot20=False
)
tracker = BoTSORT(tracker_args, frame_rate=30)
print("✅ BoT-SORT tracker initialized")

# -----------------------------
# Video Capture + Writer
# -----------------------------
cap = cv2.VideoCapture(VIDEO_PATH)
if not cap.isOpened():
    raise IOError(f"Cannot open video file: {VIDEO_PATH}")

fps = cap.get(cv2.CAP_PROP_FPS) or 30.0
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # mp4 codec
out = cv2.VideoWriter(OUTPUT_PATH, fourcc, fps, (frame_width, frame_height))

# -----------------------------
# Processing loop
# -----------------------------
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # YOLOv8 detection
    results = model.predict(
        frame, imgsz=640, conf=CONF_THRES, iou=IOU_THRES,
        classes=[PIPE_ID], device=device, half=half, verbose=False
    )[0]

    # Build detections array [x1, y1, x2, y2, score, class]
    dets = []
    for box in results.boxes:
        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
        score = float(box.conf.cpu().numpy())
        cls = int(box.cls.cpu().numpy())
        dets.append([x1, y1, x2, y2, score, cls])
    dets_np = np.array(dets) if len(dets) > 0 else np.empty((0, 6))

    # Update tracker
    tracks = tracker.update(dets_np, frame)

    # Draw tracks
    for t in tracks:
        x1, y1, x2, y2, track_id = map(int, t[:5])
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f"ID {track_id}", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    # Save frame to output
    out.write(frame)

# -----------------------------
# Release resources
# -----------------------------
cap.release()
out.release()
print(f"🎉 Done! Output video saved at: {OUTPUT_PATH}")


✅ YOLOv8 model loaded on cpu
✅ BoT-SORT tracker initialized
🎉 Done! Output video saved at: output.mp4


In [ ]:
#!/usr/bin/env python3
"""
BoT-SORT Pipeline Tracking with Smoothed Red Line
"""

from types import SimpleNamespace
from collections import deque
import math
import numpy as np
import cv2
from ultralytics import YOLO
sys.path.append("botsort")
from botsort.bot_sort import BoTSORT

# -----------------------------
# CONFIGURATION
# -----------------------------
VIDEO_PATH = "WhatsApp Vidéo 2025-07-18 à 16.23.28_d90beaab.mp4"       # Input video path
OUTPUT_PATH = "tracked_Bot2.mp4"     # Output video path
YOLO_WEIGHTS = "/content/best (2) (1).pt"           # YOLOv8 weights path
PIPE_ID = 0                           # Pipeline class ID (adjust to your dataset)
DEVICE = "cuda" if cv2.cuda.getCudaEnabledDeviceCount() > 0 else "cpu"

# -----------------------------
# INITIALIZE YOLOv8 MODEL
# -----------------------------
model = YOLO(YOLO_WEIGHTS).to(DEVICE)

# -----------------------------
# BoT-SORT tracker setup
# -----------------------------
tracker_args = SimpleNamespace(
    track_high_thresh=0.5,
    track_low_thresh=0.1,
    new_track_thresh=0.7,
    track_buffer=30,
    match_thresh=0.8,
    use_reid=False,
    with_gmc=True,
    gmc_method="orb",
    proximity_thresh=50,
    cmc_method="orb"
)

tracker = BoTSORT(args=tracker_args, frame_rate=30)
print("✅ BoT-SORT tracker initialized")

# -----------------------------
# VIDEO I/O
# -----------------------------
cap = cv2.VideoCapture(VIDEO_PATH)
W, H = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS) or 30.0
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
writer = cv2.VideoWriter(OUTPUT_PATH, fourcc, fps, (W, H))

# -----------------------------
# TRAJECTORY STORAGE
# -----------------------------
trajectory_history = deque(maxlen=10)
angle_history = deque(maxlen=10)
frame_data = []

frame_count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame_count += 1

    # YOLOv8 detection
    results = model.predict(
        frame, imgsz=960, conf=0.25, iou=0.5,
        classes=[PIPE_ID], device=DEVICE, verbose=False
    )[0]

    # Build detections: [x1, y1, x2, y2, score]
    dets = []
    if results.boxes is not None:
        xyxy = results.boxes.xyxy.cpu().numpy()
        conf = results.boxes.conf.cpu().numpy()
        for (x1, y1, x2, y2), c in zip(xyxy, conf):
            dets.append([x1, y1, x2, y2, c])

    dets_np = np.array(dets, dtype=np.float32) if len(dets) else np.empty((0, 5), dtype=np.float32)
    if dets_np.shape[0] > 0:
        class_col = np.zeros((dets_np.shape[0], 1), dtype=np.float32)  # single class = 0
        dets_np = np.hstack((dets_np, class_col))

    # Update tracker
    tracks = tracker.update(dets_np, frame)

    # --- Compute mid-lines and smoothed trajectory ---
    frame_mid_lines = []
    for t in tracks:
        x1, y1, x2, y2 = map(int, t.tlbr)
        tid = int(t.track_id)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f"pipeline#{tid}", (x1, max(0, y1-7)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        # --- Compute mid-line ---
        if (x2 - x1) > (y2 - y1):
            mid_y = (y1 + y2) // 2
            start_point = (x1, mid_y)
            end_point = (x2, mid_y)
        else:
            mid_x = (x1 + x2) // 2
            start_point = (mid_x, y1)
            end_point = (mid_x, y2)

        cv2.line(frame, start_point, end_point, (255, 255, 255), 2)
        frame_mid_lines.append((start_point, end_point))

    # --- Compute smoothed trajectory ---
    if frame_mid_lines:
        avg_start = (
            int(sum(pt[0][0] for pt in frame_mid_lines) / len(frame_mid_lines)),
            int(sum(pt[0][1] for pt in frame_mid_lines) / len(frame_mid_lines))
        )
        avg_end = (
            int(sum(pt[1][0] for pt in frame_mid_lines) / len(frame_mid_lines)),
            int(sum(pt[1][1] for pt in frame_mid_lines) / len(frame_mid_lines))
        )

        trajectory_history.append((avg_start, avg_end))

        smoothed_start = (
            int(sum(pt[0][0] for pt in trajectory_history) / len(trajectory_history)),
            int(sum(pt[0][1] for pt in trajectory_history) / len(trajectory_history))
        )
        smoothed_end = (
            int(sum(pt[1][0] for pt in trajectory_history) / len(trajectory_history)),
            int(sum(pt[1][1] for pt in trajectory_history) / len(trajectory_history))
        )

        dx = smoothed_end[0] - smoothed_start[0]
        dy = smoothed_end[1] - smoothed_start[1]
        angle_deg = math.degrees(math.atan2(dy, dx))
        angle_history.append(angle_deg)
        smoothed_angle = sum(angle_history) / len(angle_history)

        cv2.line(frame, smoothed_start, smoothed_end, (0, 0, 255), 3)
        frame_data.append((smoothed_start, smoothed_end, dx, dy, smoothed_angle))

        print(f"Frame {frame_count}: Start={smoothed_start}, End={smoothed_end}, Δx={dx}, Δy={dy}, Angle={smoothed_angle:.2f}°")

cap.release()
writer.release()
print(f"🎉 Done! Output saved at: {OUTPUT_PATH}")



✅ BoT-SORT tracker initialized
🎉 Done! Output saved at: tracked_Bot2.mp4



##Last Update

In [ ]:
#!/usr/bin/env python3
"""
BoT-SORT Pipeline Tracking with Smoothed Red Line
"""

import sys
from types import SimpleNamespace
from collections import deque
import math
import numpy as np
import cv2
from ultralytics import YOLO

# Add BoT-SORT path
sys.path.append("botsort")
from botsort.bot_sort import BoTSORT

# -----------------------------
# CONFIGURATION
# -----------------------------
VIDEO_PATH = "/content/WhatsApp Vidéo 2025-07-18 à 16.23.28_d90beaab.mp4"       # Input video path
OUTPUT_PATH = "tracked_Bot_new1.mp4"     # Output video path
YOLO_WEIGHTS = "best (5) (2).pt"           # YOLOv8 weights path
PIPE_ID = 0                           # Pipeline class ID (adjust to your dataset)

# -----------------------------
# DEVICE SETUP
# -----------------------------
device = "cuda" if cv2.cuda.getCudaEnabledDeviceCount() > 0 else "cpu"
half = (device == "cuda")

# -----------------------------
# LOAD YOLOv8 MODEL
# -----------------------------
model = YOLO(YOLO_WEIGHTS).to(device)
print(f"✅ YOLOv8 model loaded on {device}")

# -----------------------------
# INITIALIZE BoT-SORT TRACKER
# -----------------------------
tracker_args = SimpleNamespace(
    track_high_thresh=0.5,
    track_low_thresh=0.1,
    new_track_thresh=0.7,
    track_buffer=30,
    match_thresh=0.8,
    use_reid=False,
    with_gmc=False,           # disable GMC to avoid warnings & speed up
    gmc_method="orb",
    proximity_thresh=50,
    appearance_thresh=0.25,
    cmc_method="sparseOptFlow",
    frame_rate=30,
    lambda_=0.98,
    mot20=False
)
tracker = BoTSORT(tracker_args, frame_rate=30)
print("✅ BoT-SORT tracker initialized")

# -----------------------------
# VIDEO CAPTURE & WRITER
# -----------------------------
cap = cv2.VideoCapture(VIDEO_PATH)
ret, frame = cap.read()
if not ret:
    raise IOError(f"Cannot read video file: {VIDEO_PATH}")

H, W = frame.shape[:2]
fps = cap.get(cv2.CAP_PROP_FPS) or 30.0
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
writer = cv2.VideoWriter(OUTPUT_PATH, fourcc, fps, (W, H))

# -----------------------------
# TRAJECTORY STORAGE
# -----------------------------
trajectory_history = deque(maxlen=10)
angle_history = deque(maxlen=10)

frame_count = 0

# -----------------------------
# PROCESS VIDEO
# -----------------------------
while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame_count += 1

    # YOLOv8 detection
    results = model.predict(
        frame, imgsz=640, conf=0.25, iou=0.5,
        classes=[PIPE_ID], device=device, half=half, verbose=False
    )[0]

    # Build detections array [x1, y1, x2, y2, score, class]
    dets = []
    if results.boxes is not None and len(results.boxes) > 0:
        xyxy = results.boxes.xyxy.cpu().numpy()
        conf = results.boxes.conf.cpu().numpy()
        for (x1, y1, x2, y2), c in zip(xyxy, conf):
            dets.append([x1, y1, x2, y2, c])
    dets_np = np.array(dets, dtype=np.float32) if len(dets) > 0 else np.empty((0,5), dtype=np.float32)
    if dets_np.shape[0] > 0:
        class_col = np.zeros((dets_np.shape[0],1), dtype=np.float32)
        dets_np = np.hstack((dets_np, class_col))

    # Update tracker
    tracks = tracker.update(dets_np, frame)
    frame_mid_lines = []

    for t in tracks:
        x1, y1, x2, y2 = map(int, t.tlbr)
        tid = int(t.track_id)
        # Draw bounding box
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f"ID {tid}", (x1, max(0, y1-7)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        # Compute mid-line
        if (x2 - x1) > (y2 - y1):
            mid_y = (y1 + y2) // 2
            start_point = (x1, mid_y)
            end_point = (x2, mid_y)
        else:
            mid_x = (x1 + x2) // 2
            start_point = (mid_x, y1)
            end_point = (mid_x, y2)
        cv2.line(frame, start_point, end_point, (255, 255, 255), 2)
        frame_mid_lines.append((start_point, end_point))

    # Compute smoothed trajectory if any detections exist
    if frame_mid_lines:
        avg_start = (
            int(np.mean([p[0][0] for p in frame_mid_lines])),
            int(np.mean([p[0][1] for p in frame_mid_lines]))
        )
        avg_end = (
            int(np.mean([p[1][0] for p in frame_mid_lines])),
            int(np.mean([p[1][1] for p in frame_mid_lines]))
        )
        trajectory_history.append((avg_start, avg_end))
        smoothed_start = (
            int(np.mean([p[0][0] for p in trajectory_history])),
            int(np.mean([p[0][1] for p in trajectory_history]))
        )
        smoothed_end = (
            int(np.mean([p[1][0] for p in trajectory_history])),
            int(np.mean([p[1][1] for p in trajectory_history]))
        )

        # Draw smoothed red line
        cv2.line(frame, smoothed_start, smoothed_end, (0, 0, 255), 3)

    # Save frame
    writer.write(frame)

cap.release()
writer.release()
print(f"🎉 Done! Output saved at: {OUTPUT_PATH}")


✅ YOLOv8 model loaded on cpu
✅ BoT-SORT tracker initialized
🎉 Done! Output saved at: tracked_Bot_new1.mp4


In [ ]:
#!/usr/bin/env python3
"""
BoT-SORT Pipeline Tracking with Smoothed Red Line
With console output for progress
"""

import sys
from types import SimpleNamespace
from collections import deque
import math
import numpy as np
import cv2
from ultralytics import YOLO

# Add BoT-SORT path
sys.path.append("botsort")
from botsort.bot_sort import BoTSORT

# -----------------------------
# CONFIGURATION
# -----------------------------
VIDEO_PATH = "/content/WhatsApp Vidéo 2025-07-18 à 16.23.28_d90beaab.mp4"       # Input video path
OUTPUT_PATH = "tracked_Bot_new2.mp4"     # Output video path
YOLO_WEIGHTS = "/content/best (5) (2).pt"           # YOLOv8 weights path
PIPE_ID = 0                           # Pipeline class ID (adjust to your dataset)

# -----------------------------
# DEVICE SETUP
# -----------------------------
device = "cuda" if cv2.cuda.getCudaEnabledDeviceCount() > 0 else "cpu"
half = (device == "cuda")

# -----------------------------
# LOAD YOLOv8 MODEL
# -----------------------------
model = YOLO(YOLO_WEIGHTS).to(device)
print(f"✅ YOLOv8 model loaded on {device}")

# -----------------------------
# INITIALIZE BoT-SORT TRACKER
# -----------------------------
tracker_args = SimpleNamespace(
    track_high_thresh=0.5,
    track_low_thresh=0.1,
    new_track_thresh=0.7,
    track_buffer=30,
    match_thresh=0.8,
    use_reid=False,
    with_gmc=False,           # disable GMC to avoid warnings & speed up
    gmc_method="orb",
    proximity_thresh=50,
    appearance_thresh=0.25,
    cmc_method="sparseOptFlow",
    frame_rate=30,
    lambda_=0.98,
    mot20=False
)
tracker = BoTSORT(tracker_args, frame_rate=30)
print("✅ BoT-SORT tracker initialized")

# -----------------------------
# VIDEO CAPTURE & WRITER
# -----------------------------
cap = cv2.VideoCapture(VIDEO_PATH)
ret, frame = cap.read()
if not ret:
    raise IOError(f"Cannot read video file: {VIDEO_PATH}")

H, W = frame.shape[:2]
fps = cap.get(cv2.CAP_PROP_FPS) or 30.0
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
writer = cv2.VideoWriter(OUTPUT_PATH, fourcc, fps, (W, H))

# -----------------------------
# TRAJECTORY STORAGE
# -----------------------------
trajectory_history = deque(maxlen=10)
angle_history = deque(maxlen=10)

frame_count = 0

# -----------------------------
# PROCESS VIDEO
# -----------------------------
while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame_count += 1

    # YOLOv8 detection
    results = model.predict(
        frame, imgsz=640, conf=0.25, iou=0.5,
        classes=[PIPE_ID], device=device, half=half, verbose=False
    )[0]

    # Build detections array [x1, y1, x2, y2, score, class]
    dets = []
    if results.boxes is not None and len(results.boxes) > 0:
        xyxy = results.boxes.xyxy.cpu().numpy()
        conf = results.boxes.conf.cpu().numpy()
        for (x1, y1, x2, y2), c in zip(xyxy, conf):
            dets.append([x1, y1, x2, y2, c])
    dets_np = np.array(dets, dtype=np.float32) if len(dets) > 0 else np.empty((0,5), dtype=np.float32)
    if dets_np.shape[0] > 0:
        class_col = np.zeros((dets_np.shape[0],1), dtype=np.float32)
        dets_np = np.hstack((dets_np, class_col))

    # Update tracker
    tracks = tracker.update(dets_np, frame)
    frame_mid_lines = []

    for t in tracks:
        x1, y1, x2, y2 = map(int, t.tlbr)
        tid = int(t.track_id)
        # Draw bounding box
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f"ID {tid}", (x1, max(0, y1-7)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        # Compute mid-line
        if (x2 - x1) > (y2 - y1):
            mid_y = (y1 + y2) // 2
            start_point = (x1, mid_y)
            end_point = (x2, mid_y)
        else:
            mid_x = (x1 + x2) // 2
            start_point = (mid_x, y1)
            end_point = (mid_x, y2)
        cv2.line(frame, start_point, end_point, (255, 255, 255), 2)
        frame_mid_lines.append((start_point, end_point))

    # Compute smoothed trajectory if any detections exist
    if frame_mid_lines:
        avg_start = (
            int(np.mean([p[0][0] for p in frame_mid_lines])),
            int(np.mean([p[0][1] for p in frame_mid_lines]))
        )
        avg_end = (
            int(np.mean([p[1][0] for p in frame_mid_lines])),
            int(np.mean([p[1][1] for p in frame_mid_lines]))
        )
        trajectory_history.append((avg_start, avg_end))
        smoothed_start = (
            int(np.mean([p[0][0] for p in trajectory_history])),
            int(np.mean([p[0][1] for p in trajectory_history]))
        )
        smoothed_end = (
            int(np.mean([p[1][0] for p in trajectory_history])),
            int(np.mean([p[1][1] for p in trajectory_history]))
        )

        # Draw smoothed red line
        cv2.line(frame, smoothed_start, smoothed_end, (0, 0, 255), 3)

        # Console output
        print(f"[Frame {frame_count}] Tracks: {len(tracks)}, "
              f"Smoothed line: {smoothed_start} -> {smoothed_end}")

    else:
        print(f"[Frame {frame_count}] No tracks detected")

    # Save frame
    writer.write(frame)

cap.release()
writer.release()
print(f"🎉 Done! Output saved at: {OUTPUT_PATH}")


✅ YOLOv8 model loaded on cpu
✅ BoT-SORT tracker initialized
[Frame 1] Tracks: 1, Smoothed line: (210, 98) -> (210, 595)
[Frame 2] Tracks: 1, Smoothed line: (209, 96) -> (209, 593)
[Frame 3] Tracks: 1, Smoothed line: (208, 94) -> (208, 593)
[Frame 4] Tracks: 1, Smoothed line: (207, 93) -> (207, 593)
[Frame 5] Tracks: 1, Smoothed line: (206, 92) -> (206, 592)
[Frame 6] Tracks: 1, Smoothed line: (206, 91) -> (206, 591)
[Frame 7] Tracks: 1, Smoothed line: (206, 90) -> (206, 591)
[Frame 8] Tracks: 1, Smoothed line: (205, 89) -> (205, 591)
[Frame 9] Tracks: 1, Smoothed line: (205, 89) -> (205, 591)
[Frame 10] Tracks: 1, Smoothed line: (204, 88) -> (204, 590)
[Frame 11] Tracks: 1, Smoothed line: (203, 87) -> (203, 590)
[Frame 12] Tracks: 1, Smoothed line: (203, 86) -> (203, 589)
[Frame 13] Tracks: 1, Smoothed line: (202, 86) -> (202, 589)
[Frame 14] Tracks: 1, Smoothed line: (202, 85) -> (202, 588)
[Frame 15] Tracks: 1, Smoothed line: (201, 85) -> (201, 588)
[Frame 16] Tracks: 1, Smoothed lin

In [ ]:
#!/usr/bin/env python3
"""
Optimized BoT-SORT Pipeline Tracking with Smoothed Red Line
"""

import sys
import math
from types import SimpleNamespace
from collections import deque
import numpy as np
import cv2
import torch
from ultralytics import YOLO

# -----------------------------
# CONFIGURATION
# -----------------------------
VIDEO_PATH = "/content/WhatsApp Vidéo 2025-07-18 à 16.23.28_d90beaab.mp4"            # input video path
OUTPUT_PATH = "tracked_Bot4_fast.mp4"    # output video path
YOLO_WEIGHTS = "/content/best (5) (2).pt"               # YOLOv8 weights
PIPE_ID = 0                               # pipeline class ID
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
HALF_PRECISION = DEVICE == "cuda"         # use FP16 on GPU
FRAME_SKIP = 1                             # skip frames to speed up (1 = no skip)
TRAJECTORY_LEN = 10                        # smoothing window

# -----------------------------
# LOAD YOLOv8 MODEL
# -----------------------------
model = YOLO(YOLO_WEIGHTS).to(DEVICE)
print(f"✅ YOLOv8 model loaded on {DEVICE}")

# -----------------------------
# IMPORT BoT-SORT
# -----------------------------
sys.path.append("botsort")  # path to BoT-SORT repo
from botsort.bot_sort import BoTSORT

# -----------------------------
# INITIALIZE BoT-SORT TRACKER
# -----------------------------
tracker_args = SimpleNamespace(
    track_high_thresh=0.5,
    track_low_thresh=0.1,
    new_track_thresh=0.7,
    track_buffer=20,
    match_thresh=0.8,
    use_reid=False,
    with_gmc=False,       # disable for speed
    gmc_method="orb",
    proximity_thresh=50,
    cmc_method="orb"
)
tracker = BoTSORT(args=tracker_args, frame_rate=30)
print("✅ BoT-SORT tracker initialized")

# -----------------------------
# VIDEO I/O
# -----------------------------
cap = cv2.VideoCapture(VIDEO_PATH)
if not cap.isOpened():
    raise IOError(f"Cannot open video file: {VIDEO_PATH}")

W, H = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS) or 30.0
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
writer = cv2.VideoWriter(OUTPUT_PATH, fourcc, fps, (W, H))

# -----------------------------
# TRAJECTORY STORAGE
# -----------------------------
trajectory_history = deque(maxlen=TRAJECTORY_LEN)
angle_history = deque(maxlen=TRAJECTORY_LEN)

frame_count = 0

# -----------------------------
# PROCESS VIDEO
# -----------------------------
while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame_count += 1

    # Skip frames if needed
    if frame_count % FRAME_SKIP != 0:
        continue

    # --- YOLOv8 detection ---
    results = model.predict(
        frame, imgsz=640, conf=0.25, iou=0.5,
        classes=[PIPE_ID], device=DEVICE, half=HALF_PRECISION, verbose=False
    )[0]

    # --- Build detection array [x1, y1, x2, y2, score, class] ---
    dets = []
    if results.boxes is not None and len(results.boxes) > 0:
        xyxy = results.boxes.xyxy.cpu().numpy()
        conf = results.boxes.conf.cpu().numpy()
        dets = np.hstack([xyxy, conf[:, None]]).astype(np.float32)
        class_col = np.zeros((dets.shape[0], 1), dtype=np.float32)
        dets = np.hstack([dets, class_col])
    else:
        dets = np.empty((0,6), dtype=np.float32)

    # --- Update tracker ---
    tracks = tracker.update(dets, frame)
    frame_mid_lines = []

    # --- Process tracks ---
    for t in tracks:
        x1, y1, x2, y2 = map(int, t.tlbr)
        tid = int(t.track_id)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f"ID {tid}", (x1, max(0, y1-7)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        # Compute mid-line
        if (x2 - x1) > (y2 - y1):
            mid_y = (y1 + y2) // 2
            start_point, end_point = (x1, mid_y), (x2, mid_y)
        else:
            mid_x = (x1 + x2) // 2
            start_point, end_point = (mid_x, y1), (mid_x, y2)
        cv2.line(frame, start_point, end_point, (255, 255, 255), 2)
        frame_mid_lines.append((start_point, end_point))

    # --- Compute smoothed trajectory ---
    if frame_mid_lines:
        avg_start = tuple(np.mean([p[0] for p in frame_mid_lines], axis=0).astype(int))
        avg_end = tuple(np.mean([p[1] for p in frame_mid_lines], axis=0).astype(int))
        trajectory_history.append((avg_start, avg_end))

        smoothed_start = tuple(np.mean([p[0] for p in trajectory_history], axis=0).astype(int))
        smoothed_end = tuple(np.mean([p[1] for p in trajectory_history], axis=0).astype(int))

        dx, dy = smoothed_end[0] - smoothed_start[0], smoothed_end[1] - smoothed_start[1]
        angle_deg = math.degrees(math.atan2(dy, dx))
        angle_history.append(angle_deg)
        smoothed_angle = np.mean(angle_history)

        # Draw smoothed red line
        cv2.line(frame, smoothed_start, smoothed_end, (0, 0, 255), 3)

        # Console output
        print(f"[Frame {frame_count}] Tracks: {len(tracks)}, "
              f"Smoothed line: {smoothed_start} -> {smoothed_end}, Angle: {smoothed_angle:.2f}°")
    else:
        print(f"[Frame {frame_count}] No tracks detected")

    # --- Write frame to output video ---
    writer.write(frame)

cap.release()
writer.release()
print(f"🎉 Done! Output saved at: {OUTPUT_PATH}")


✅ YOLOv8 model loaded on cpu
✅ BoT-SORT tracker initialized
[Frame 1] Tracks: 1, Smoothed line: (np.int64(212), np.int64(96)) -> (np.int64(212), np.int64(594)), Angle: 90.00°
[Frame 2] Tracks: 1, Smoothed line: (np.int64(211), np.int64(97)) -> (np.int64(211), np.int64(594)), Angle: 90.00°
[Frame 3] Tracks: 1, Smoothed line: (np.int64(210), np.int64(96)) -> (np.int64(210), np.int64(593)), Angle: 90.00°
[Frame 4] Tracks: 1, Smoothed line: (np.int64(209), np.int64(95)) -> (np.int64(209), np.int64(593)), Angle: 90.00°
[Frame 5] Tracks: 1, Smoothed line: (np.int64(208), np.int64(94)) -> (np.int64(208), np.int64(593)), Angle: 90.00°
[Frame 6] Tracks: 1, Smoothed line: (np.int64(207), np.int64(93)) -> (np.int64(207), np.int64(592)), Angle: 90.00°
[Frame 7] Tracks: 1, Smoothed line: (np.int64(207), np.int64(92)) -> (np.int64(207), np.int64(591)), Angle: 90.00°
[Frame 8] Tracks: 1, Smoothed line: (np.int64(206), np.int64(91)) -> (np.int64(206), np.int64(591)), Angle: 90.00°
[Frame 9] Tracks: 1,

In [ ]:
#!/usr/bin/env python3
"""
Optimized BoT-SORT Pipeline Tracking with Continuous Smoothed Red Line
"""

import sys
import math
from types import SimpleNamespace
from collections import deque
import numpy as np
import cv2
import torch
from ultralytics import YOLO

# -----------------------------
# CONFIGURATION
# -----------------------------
VIDEO_PATH = "/content/WhatsApp Vidéo 2025-07-18 à 16.23.28_d90beaab.mp4"            # input video path
OUTPUT_PATH = "tracked_Bot4_fast_opt.mp4"    # output video path
YOLO_WEIGHTS = "/content/best (5) (2).pt"
PIPE_ID = 0                            # Pipeline class ID
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
HALF_PRECISION = DEVICE == "cuda"
FRAME_SKIP = 1                          # Process every N-th frame
TRAJECTORY_LEN = 10                     # smoothing window

# -----------------------------
# LOAD YOLOv8 MODEL
# -----------------------------
model = YOLO(YOLO_WEIGHTS).to(DEVICE)
print(f"✅ YOLOv8 model loaded on {DEVICE}")

# -----------------------------
# IMPORT BoT-SORT
# -----------------------------
sys.path.append("botsort")  # path to BoT-SORT repo
from botsort.bot_sort import BoTSORT

# -----------------------------
# INITIALIZE BoT-SORT TRACKER
# -----------------------------
tracker_args = SimpleNamespace(
    track_high_thresh=0.5,
    track_low_thresh=0.1,
    new_track_thresh=0.7,
    track_buffer=20,
    match_thresh=0.8,
    use_reid=False,
    with_gmc=False,
    gmc_method="orb",
    proximity_thresh=50,
    cmc_method="orb"
)
tracker = BoTSORT(args=tracker_args, frame_rate=30)
print("✅ BoT-SORT tracker initialized")

# -----------------------------
# VIDEO I/O
# -----------------------------
cap = cv2.VideoCapture(VIDEO_PATH)
if not cap.isOpened():
    raise IOError(f"Cannot open video file: {VIDEO_PATH}")

W, H = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS) or 30.0
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
writer = cv2.VideoWriter(OUTPUT_PATH, fourcc, fps, (W, H))

# -----------------------------
# TRAJECTORY STORAGE
# -----------------------------
trajectory_history = deque(maxlen=TRAJECTORY_LEN)
angle_history = deque(maxlen=TRAJECTORY_LEN)

last_smoothed_start = None
last_smoothed_end = None
frame_count = 0

# -----------------------------
# PROCESS VIDEO
# -----------------------------
while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame_count += 1

    if frame_count % FRAME_SKIP != 0:
        continue

    # --- YOLOv8 detection ---
    results = model.predict(
        frame, imgsz=640, conf=0.25, iou=0.5,
        classes=[PIPE_ID], device=DEVICE, half=HALF_PRECISION, verbose=False
    )[0]

    # --- Build detection array [x1, y1, x2, y2, score, class] ---
    dets = []
    if results.boxes is not None and len(results.boxes) > 0:
        xyxy = results.boxes.xyxy.cpu().numpy()
        conf = results.boxes.conf.cpu().numpy()
        dets = np.hstack([xyxy, conf[:, None]]).astype(np.float32)
        class_col = np.zeros((dets.shape[0], 1), dtype=np.float32)
        dets = np.hstack([dets, class_col])
    else:
        dets = np.empty((0, 6), dtype=np.float32)

    # --- Update tracker ---
    tracks = tracker.update(dets, frame)
    frame_mid_lines = []

    # --- Process tracks ---
    for t in tracks:
        x1, y1, x2, y2 = map(int, t.tlbr)
        tid = int(t.track_id)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f"ID {tid}", (x1, max(0, y1-7)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        if (x2 - x1) > (y2 - y1):
            mid_y = (y1 + y2) // 2
            start_point, end_point = (x1, mid_y), (x2, mid_y)
        else:
            mid_x = (x1 + x2) // 2
            start_point, end_point = (mid_x, y1), (mid_x, y2)

        cv2.line(frame, start_point, end_point, (255, 255, 255), 2)
        frame_mid_lines.append((start_point, end_point))

    # --- Compute smoothed trajectory ---
    if frame_mid_lines:
        avg_start = tuple(np.mean([p[0] for p in frame_mid_lines], axis=0).astype(int))
        avg_end = tuple(np.mean([p[1] for p in frame_mid_lines], axis=0).astype(int))
        trajectory_history.append((avg_start, avg_end))

        smoothed_start = tuple(np.mean([p[0] for p in trajectory_history], axis=0).astype(int))
        smoothed_end = tuple(np.mean([p[1] for p in trajectory_history], axis=0).astype(int))

        dx, dy = smoothed_end[0] - smoothed_start[0], smoothed_end[1] - smoothed_start[1]
        angle_deg = math.degrees(math.atan2(dy, dx))
        angle_history.append(angle_deg)
        smoothed_angle = np.mean(angle_history)

        last_smoothed_start, last_smoothed_end = smoothed_start, smoothed_end
    else:
        # No detection: use last known smoothed line
        if last_smoothed_start is not None and last_smoothed_end is not None:
            smoothed_start, smoothed_end = last_smoothed_start, last_smoothed_end
        else:
            smoothed_start = smoothed_end = None

    # Draw smoothed red line if available
    if smoothed_start and smoothed_end:
        cv2.line(frame, smoothed_start, smoothed_end, (0, 0, 255), 3)

    # --- Console output ---
    print(f"[Frame {frame_count}] Tracks: {len(tracks)}, "
          f"Smoothed line: {smoothed_start} -> {smoothed_end}" if smoothed_start else
          f"[Frame {frame_count}] Tracks: {len(tracks)}, No smoothed line yet")

    # --- Write frame ---
    writer.write(frame)

cap.release()
writer.release()
print(f"🎉 Done! Output saved at: {OUTPUT_PATH}")


✅ YOLOv8 model loaded on cpu
✅ BoT-SORT tracker initialized
[Frame 1] Tracks: 1, Smoothed line: (np.int64(212), np.int64(96)) -> (np.int64(212), np.int64(594))
[Frame 2] Tracks: 1, Smoothed line: (np.int64(211), np.int64(97)) -> (np.int64(211), np.int64(594))
[Frame 3] Tracks: 1, Smoothed line: (np.int64(210), np.int64(96)) -> (np.int64(210), np.int64(593))
[Frame 4] Tracks: 1, Smoothed line: (np.int64(209), np.int64(95)) -> (np.int64(209), np.int64(593))
[Frame 5] Tracks: 1, Smoothed line: (np.int64(208), np.int64(94)) -> (np.int64(208), np.int64(593))
[Frame 6] Tracks: 1, Smoothed line: (np.int64(207), np.int64(93)) -> (np.int64(207), np.int64(592))
[Frame 7] Tracks: 1, Smoothed line: (np.int64(207), np.int64(92)) -> (np.int64(207), np.int64(591))
[Frame 8] Tracks: 1, Smoothed line: (np.int64(206), np.int64(91)) -> (np.int64(206), np.int64(591))
[Frame 9] Tracks: 1, Smoothed line: (np.int64(206), np.int64(90)) -> (np.int64(206), np.int64(591))
[Frame 10] Tracks: 1, Smoothed line: (np

In [ ]:
#!/usr/bin/env python3
"""
BoT-SORT + YOLOv8 Pipeline Tracking with Smoothed Red Line
Supports both offline video files and live drone streams (RTSP/USB).
Always saves output to disk as MP4.
"""

import sys
import math
from types import SimpleNamespace
from collections import deque
import numpy as np
import cv2
import torch
from ultralytics import YOLO

# -----------------------------
# CONFIGURATION
# -----------------------------
# Example for file: "/content/input.mp4"
# Example for webcam: 0
# Example for drone RTSP: "rtsp://192.168.10.1:554/stream"
VIDEO_PATH = "rtsp://192.168.10.1:554/stream"

OUTPUT_PATH = "tracked_output.mp4"   # always saves result
YOLO_WEIGHTS = "/content/best (5) (2).pt"
PIPE_ID = 0                                 # pipeline class ID
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
HALF_PRECISION = DEVICE == "cuda"
FRAME_SKIP = 2                              # process every 2nd frame
TRAJECTORY_LEN = 10                         # smoothing window

# -----------------------------
# LOAD YOLOv8 MODEL
# -----------------------------
model = YOLO(YOLO_WEIGHTS).to(DEVICE)
if HALF_PRECISION:
    model.fuse()
print(f"✅ YOLOv8 model loaded on {DEVICE}")

# -----------------------------
# IMPORT BoT-SORT
# -----------------------------
sys.path.append("botsort")
from botsort.bot_sort import BoTSORT

tracker_args = SimpleNamespace(
    track_high_thresh=0.5,
    track_low_thresh=0.1,
    new_track_thresh=0.7,
    track_buffer=20,
    match_thresh=0.8,
    use_reid=False,
    with_gmc=False,
    gmc_method="orb",
    proximity_thresh=50,
    cmc_method="orb"
)
tracker = BoTSORT(args=tracker_args, frame_rate=30)
print("✅ BoT-SORT tracker initialized")

# -----------------------------
# VIDEO I/O
# -----------------------------
cap = cv2.VideoCapture(VIDEO_PATH)
if not cap.isOpened():
    raise IOError(f"❌ Cannot open video source: {VIDEO_PATH}")

W, H = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
if fps == 0 or np.isnan(fps):  # fallback for live streams
    fps = 30.0

fourcc = cv2.VideoWriter_fourcc(*"mp4v")
writer = cv2.VideoWriter(OUTPUT_PATH, fourcc, fps, (W, H))

# -----------------------------
# TRAJECTORY STORAGE
# -----------------------------
trajectory_history = deque(maxlen=TRAJECTORY_LEN)
angle_history = deque(maxlen=TRAJECTORY_LEN)

last_smoothed_start = None
last_smoothed_end = None
frame_count = 0

# -----------------------------
# PROCESS VIDEO
# -----------------------------
while True:
    ret, frame = cap.read()
    if not ret:
        print("⚠️ Frame grab failed, retrying...")
        continue  # don’t break for live streams

    frame_count += 1
    if frame_count % FRAME_SKIP != 0:
        continue

    # --- YOLOv8 detection ---
    results = model.predict(
        frame, imgsz=320, conf=0.25, iou=0.5,
        classes=[PIPE_ID], device=DEVICE, half=HALF_PRECISION, verbose=False
    )[0]

    if results.boxes is not None and len(results.boxes) > 0:
        xyxy = results.boxes.xyxy.cpu().numpy()
        conf = results.boxes.conf.cpu().numpy()
        dets = np.hstack([xyxy, conf[:, None]]).astype(np.float32)
        class_col = np.zeros((dets.shape[0], 1), dtype=np.float32)
        dets = np.hstack([dets, class_col])
    else:
        dets = np.empty((0, 6), dtype=np.float32)

    # --- Update tracker ---
    tracks = tracker.update(dets, frame)
    frame_mid_lines = []

    # --- Process tracks ---
    for t in tracks:
        x1, y1, x2, y2 = map(int, t.tlbr)
        tid = int(t.track_id)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f"ID {tid}", (x1, max(0, y1 - 7)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        if (x2 - x1) > (y2 - y1):  # horizontal
            mid_y = (y1 + y2) // 2
            start_point, end_point = (x1, mid_y), (x2, mid_y)
        else:  # vertical
            mid_x = (x1 + x2) // 2
            start_point, end_point = (mid_x, y1), (mid_x, y2)

        cv2.line(frame, start_point, end_point, (255, 255, 255), 2)
        frame_mid_lines.append((start_point, end_point))

    # --- Compute smoothed trajectory ---
    if frame_mid_lines:
        avg_start = tuple(np.mean([p[0] for p in frame_mid_lines], axis=0).astype(int))
        avg_end = tuple(np.mean([p[1] for p in frame_mid_lines], axis=0).astype(int))
        trajectory_history.append((avg_start, avg_end))

        smoothed_start = tuple(np.mean([p[0] for p in trajectory_history], axis=0).astype(int))
        smoothed_end = tuple(np.mean([p[1] for p in trajectory_history], axis=0).astype(int))

        dx, dy = smoothed_end[0] - smoothed_start[0], smoothed_end[1] - smoothed_start[1]
        angle_deg = math.degrees(math.atan2(dy, dx))
        angle_history.append(angle_deg)
        smoothed_angle = np.mean(angle_history)

        last_smoothed_start, last_smoothed_end = smoothed_start, smoothed_end
    else:
        smoothed_start, smoothed_end = last_smoothed_start, last_smoothed_end

    if smoothed_start and smoothed_end:
        cv2.line(frame, smoothed_start, smoothed_end, (0, 0, 255), 3)

    # Console output
    print(f"[Frame {frame_count}] Tracks: {len(tracks)}, "
          f"Smoothed line: {smoothed_start} -> {smoothed_end}" if smoothed_start else
          f"[Frame {frame_count}] Tracks: {len(tracks)}, No smoothed line yet")

    # --- Save and display ---
    writer.write(frame)
    cv2.imshow("Drone Tracking", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
writer.release()
cv2.destroyAllWindows()
print(f"🎉 Done! Output video saved at: {OUTPUT_PATH}")
